In [1]:
# ── Architecture-diagram generator ──────────────────────────────────
import matplotlib; matplotlib.use("Agg")          # keep it non-interactive
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os, textwrap, json

SAVE_DIR = "outputs/architectures"                # ← change if you like
os.makedirs(SAVE_DIR, exist_ok=True)

PALETTE = ["#4F6D7A", "#C0D6DF", "#DDB967",       # pleasant muted tones
           "#95B8D1", "#ECC8AF", "#C7EFCF",
           "#F95738", "#F4A259", "#2E4057"]

def draw_arch(name, layers, palette=PALETTE, save_dir=SAVE_DIR,
              box_w=2.2, box_h=1.2, h_gap=0.6):
    """Render a left-to-right chain of labelled blocks & save as PNG."""
    fig_w = 1 + len(layers) * (box_w + h_gap)
    fig, ax = plt.subplots(figsize=(fig_w, 2.6))
    ax.axis("off")

    for i, layer in enumerate(layers):
        x = 0.5 + i * (box_w + h_gap)
        rect = patches.FancyBboxPatch(
            (x, 0.7), box_w, box_h,
            boxstyle="round,pad=0.3",
            facecolor=palette[i % len(palette)], edgecolor="black", lw=1.5
        )
        ax.add_patch(rect)
        ax.text(
            x + box_w/2, 0.7 + box_h/2,
            textwrap.fill(layer, 15),
            ha="center", va="center", fontsize=8, color="white"
        )
        if i < len(layers)-1:                    # arrow to next block
            ax.annotate("", xy=(x+box_w, 0.7+box_h/2),
                        xytext=(x+box_w+h_gap-0.1, 0.7+box_h/2),
                        arrowprops=dict(arrowstyle="->", lw=1.3))

    fig.suptitle(f"{name} Architecture", fontsize=12, fontweight="bold")
    path = os.path.join(save_dir, f"{name.lower().replace('/','_')}_architecture.png")
    plt.tight_layout(); plt.savefig(path, dpi=160, bbox_inches="tight")
    plt.show(); plt.close(fig)
    return path

# ---------- declare the layer sequences (edit freely!) --------------
architectures = {
    "GRU-LSTM": ["Input (window)", "GRU", "LSTM", "Dense / FC", "H-step forecast"],
    "SARIMA"  : ["Raw series", "Differencing", "AR part", "MA part",
                 "Seasonal (24 h)", "Linear combiner", "Forecast"],
    "N-BEATS" : ["Input", "Trend block(s)", "Seasonal block(s)",
                 "Generic block(s)", "Add-back residual", "Forecast"],
    "TiDE"    : ["Input", "Temporal Gate", "Position-wise MLP Encoder",
                 "Decoder", "Forecast"],
    "PatchTST": ["Input", "Patch embedding", "Transformer encoders × N",
                 "MLP head", "Forecast"],
    "XGBoost" : ["Windowed features", "Gradient-boosted trees", "Forecast"],
}

# ---------- generate & show -----------------------------------------
generated = {name: draw_arch(name, layers) for name, layers in architectures.items()}
print("✓ PNGs written:", json.dumps(generated, indent=2))

/tmp/ipykernel_4176710/861982921.py:41: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout(); plt.savefig(path, dpi=160, bbox_inches="tight")


✓ PNGs written: {
  "GRU-LSTM": "outputs/architectures/gru-lstm_architecture.png",
  "SARIMA": "outputs/architectures/sarima_architecture.png",
  "N-BEATS": "outputs/architectures/n-beats_architecture.png",
  "TiDE": "outputs/architectures/tide_architecture.png",
  "PatchTST": "outputs/architectures/patchtst_architecture.png",
  "XGBoost": "outputs/architectures/xgboost_architecture.png"
}
